In [1]:
!curl https://download.mosek.com/stable/8.1.0.82/mosektoolslinux64x86.tar.bz2 --output /root/mosektoolslinux64x86.tar.bz2
!cd /root/; tar xvf /root/mosektoolslinux64x86.tar.bz2
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/Mosek/mosek.lic /root/mosek/
!pip install git+https://github.com/convexengineering/gpkit
!pip show gpkit

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.3M  100 40.3M    0     0  39.6M      0  0:00:01  0:00:01 --:--:-- 39.6M
mosek/8/tools/examples/python/parameters.py
mosek/8/toolbox/examples/cqo1.m
mosek/8/toolbox/r2014a/mskqpopt.m
mosek/8/tools/examples/fusion/java/breaksolver.java
mosek/8/toolbox/r2014a/mskcheck.m
mosek/8/tools/examples/fusion/cxx/lo1.cc
mosek/8/tools/examples/data/dgo.f
mosek/8/tools/examples/dotnet/scopt1.cs
mosek/8/tools/examples/dotnet/solvebasis.cs
mosek/8/toolbox/r2014aom/mskerrmsg.m
mosek/8/toolbox/examples/simple.m
mosek/8/tools/examples/java/solutionquality.java
mosek/8/tools/examples/fusion/dotnet/production.cs
mosek/8/tools/examples/fusion/java/Makefile
mosek/8/tools/examples/fusion/python/lo1.py
mosek/8/tools/examples/fusion/java/duality.java
mosek/8/tools/platform/linux64x86/bin/libcilkrts.so.5
mosek/8/toolbox/examples/qo2.m
mosek/8/tools/ex

In [2]:
from pylab import *
from gpkit import VectorVariable, Variable, Model, units

Found no installed solvers, beginning a build.
# Building GPkit version 1.1
# Moving to the directory from which GPkit was imported.

Attempting to find and build solvers:

# Looking for `mosek_cli`
#   (A "success" is if mskexpopt complains that
#    we haven't specified a file for it to open.)
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##
# Looks like `mskexpopt` was not found in the default PATH,
#  so let's try locating that binary ourselves.
#   Adding /root/mosek/8/tools/platform/linux64x86/bin to the PATH
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##

Found mosek_cli in /root/mosek/8/tools/platform/linux64x86/bin

# Looking for `mosek_conif`
#   Trying to import mosek...
# Did not find
# mosek_conif

# Looking for `cvxopt`
#   Trying to import cvxopt...


.


Found cvxopt in the default PYTHONPATH

Found the following solvers: mosek_cli, cvxopt
#     Replacing directory env


.........................E......................................................................................................................................................................................
ERROR: test_vector_sweep (gpkit.tests.t_sub.TestModelSubs.test_vector_sweep)
Test sweep involving VectorVariables
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gpkit/tests/t_sub.py", line 205, in test_vector_sweep
    sol = m.solve(verbosity=0)
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gpkit/constraints/prog_factories.py", line 123, in solvefn
    constants, sweep, linked = parse_subs(self.varkeys, self.substitutions)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gpkit/nomials/substitution.py", line 25, in parse_subs
    append_sub(sub, keys, constants, sweep, l


GPkit is now installed with solver(s) ['mosek_cli', 'cvxopt']
To incorporate new solvers at a later date, run `gpkit.build()`.

If any tests didn't pass, please post the output above
(starting from "Found no installed solvers, beginning a build.")
to gpkit@mit.edu or https://github.com/convexengineering/gpkit/issues/new
so we can prevent others from having these errors.

The same goes for any other bugs you encounter with GPkit:
send 'em our way, along with any interesting models, speculative features,
comments, discussions, or clarifications you feel like sharing.

Finally, we hope you find our documentation (https://gpkit.readthedocs.io/)
and engineering-design models (https://github.com/convexengineering/gplibrary/)
to be useful resources for your own applications.

Enjoy!



# example 1 from the demo
size of the first gate is not fixed

In [12]:
# NOTE: This is the example from the demo - reproducing those results first
# Number of gates
N = 5

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 100

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (c_load / gate_sizes[4]) + p[4])

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(my_T, constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_wall = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_wall = {round(T_wall, 2)}')
print(f'T_wall (ps) = {round(T_wall * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 6 free variables
  in 12 posynomial inequalities.
Solving took 0.0275 seconds.
T_wall = 10.0
T_wall (ps) = 50.0
Gate Sizes: [99.9999814  99.98068327 99.97268513 99.97268513 99.98068327]


# example 2 from the demo
size of the first gate is fixed to 1, since we don't know what the max for that is, based on it's driver ckt.

In [13]:
# NOTE: This is the example from the demo - reproducing those results first
# Number of gates
N = 5

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 100

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (c_load / gate_sizes[4]) + p[4])

# additional constraint for the first gate to be minimum-sized, since we don't
# know what is the max allowed size for the first gate, depending on it's driver
constraints.append(gate_sizes[0] == 1)

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(my_T, constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_wall = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_wall = {round(T_wall, 2)}')
print(f'T_wall (ps) = {round(T_wall * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 6 free variables
  in 14 posynomial inequalities.
Solving took 0.0227 seconds.
T_wall = 17.56
T_wall (ps) = 87.8
Gate Sizes: [ 1.          2.51188634  6.30957298 15.84893016 39.81071113]


# trying stuff out from the problem statement in my assignment
again, first gate can be arbitrarily upsized.

In [16]:
# Number of gates
N = 6

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 64

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (g[5] * gate_sizes[5] / gate_sizes[4]) + p[4] + \
                    (c_load / gate_sizes[5]) + p[5])

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(my_T, constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_wall = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_wall = {round(T_wall, 2)}')
print(f'T_wall (ps) = {round(T_wall * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 7 free variables
  in 14 posynomial inequalities.
Solving took 0.026 seconds.
T_wall = 12.56
T_wall (ps) = 62.81
Gate Sizes: [63.99999466 63.98930756 63.98489245 63.98489245 63.98924357 63.99999466]


# SOlution (q1)
this one is basically the solution to the assignment (q1)

In [18]:
# Number of gates
N = 6

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 64

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (g[5] * gate_sizes[5] / gate_sizes[4]) + p[4] + \
                    (c_load / gate_sizes[5]) + p[5])

# additional constraint for the first gate to be minimum-sized, since we don't
# know what is the max allowed size for the first gate, depending on it's driver
constraints.append(gate_sizes[0] == 1)

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(my_T, constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_wall = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_wall = {round(T_wall, 2)}')
print(f'T_wall (ps) = {round(T_wall * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 7 free variables
  in 16 posynomial inequalities.
Solving took 0.0399 seconds.
T_wall = 18.93
T_wall (ps) = 94.63
Gate Sizes: [ 1.          2.15443477  4.64159009  9.99999907 21.54433705 46.41589661]


# solutuon (q2)
this is the solution to the q2 in my assignment

In [19]:
# Number of gates
N = 6

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 64

# timing specification -- this needs the T_wall obtained from the prev soln
t_spec = 1.2 * T_wall

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (g[5] * gate_sizes[5] / gate_sizes[4]) + p[4] + \
                    (c_load / gate_sizes[5]) + p[5])

# additional constraint for the first gate to be minimum-sized, since we don't
# know what is the max allowed size for the first gate, depending on it's driver
constraints.append(gate_sizes[0] == 1)

# constraint for my_T to be below the t_spec
constraints.append(my_T <= t_spec)

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(np.sum(gate_sizes), constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_arrival = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_arrival = {round(T_arrival, 2)}')
print(f'T_arrival (ps) = {round(T_arrival * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 7 free variables
  in 17 posynomial inequalities.
Solving took 0.0182 seconds.
T_arrival = 22.71
T_arrival (ps) = 113.56
Gate Sizes: [ 1.          1.00000028  1.19655477  1.86997414  3.99270122 13.4045473 ]


# trying 1.0 * twall

In [20]:
# Number of gates
N = 6

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 64

# timing specification -- this needs the T_wall obtained from the prev soln
t_spec = 1.0 * T_wall

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (g[5] * gate_sizes[5] / gate_sizes[4]) + p[4] + \
                    (c_load / gate_sizes[5]) + p[5])

# additional constraint for the first gate to be minimum-sized, since we don't
# know what is the max allowed size for the first gate, depending on it's driver
constraints.append(gate_sizes[0] == 1)

# constraint for my_T to be below the t_spec
constraints.append(my_T <= t_spec)

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(np.sum(gate_sizes), constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_arrival = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_arrival = {round(T_arrival, 2)}')
print(f'T_arrival (ps) = {round(T_arrival * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 7 free variables
  in 17 posynomial inequalities.
Solving took 0.0183 seconds.
Solution check warning: Dual: sum of nu^T * A did not vanish.
T_arrival = 18.93
T_arrival (ps) = 94.63
Gate Sizes: [ 1.          2.15295841  4.63550432  9.98192542 21.50066791 46.3390955 ]


# trying 1.3*twall

In [21]:
# Number of gates
N = 6

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 64

# timing specification -- this needs the T_wall obtained from the prev soln
t_spec = 1.3 * T_wall

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (g[5] * gate_sizes[5] / gate_sizes[4]) + p[4] + \
                    (c_load / gate_sizes[5]) + p[5])

# additional constraint for the first gate to be minimum-sized, since we don't
# know what is the max allowed size for the first gate, depending on it's driver
constraints.append(gate_sizes[0] == 1)

# constraint for my_T to be below the t_spec
constraints.append(my_T <= t_spec)

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(np.sum(gate_sizes), constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_arrival = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_arrival = {round(T_arrival, 2)}')
print(f'T_arrival (ps) = {round(T_arrival * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 7 free variables
  in 17 posynomial inequalities.
Solving took 0.0191 seconds.
T_arrival = 24.6
T_arrival (ps) = 123.02
Gate Sizes: [ 1.          1.00000005  1.00000057  1.36382343  2.87556449 10.57772103]


# trying 1.4 * twall

In [22]:
# Number of gates
N = 6

# Vector of gate sizes
gate_sizes = VectorVariable(N, "gs")

# we want to setup a variable that is going to limit the total path delay.
# by minimizing this variable, we are inherently minimizing the path delay.
my_T = Variable("my_T")

# we assume that the logical and parasitic efforts are ideal, as per the problem
# statement, for all inverters in the chain.
g = [1] * N
p = [1] * N

# capacitance that the path needs to drive (in C)
c_load = 100

# intrinsic delay of the inverter
tau = 5

# largest gate that is available to us
gate_size_max = 64

# timing specification -- this needs the T_wall obtained from the prev soln
t_spec = 1.4 * T_wall

# constraints for the solver
constraints = []
constraints.append(my_T >=
                    (g[1] * gate_sizes[1] / gate_sizes[0]) + p[0] + \
                    (g[2] * gate_sizes[2] / gate_sizes[1]) + p[1] + \
                    (g[3] * gate_sizes[3] / gate_sizes[2]) + p[2] + \
                    (g[4] * gate_sizes[4] / gate_sizes[3]) + p[3] + \
                    (g[5] * gate_sizes[5] / gate_sizes[4]) + p[4] + \
                    (c_load / gate_sizes[5]) + p[5])

# additional constraint for the first gate to be minimum-sized, since we don't
# know what is the max allowed size for the first gate, depending on it's driver
constraints.append(gate_sizes[0] == 1)

# constraint for my_T to be below the t_spec
constraints.append(my_T <= t_spec)

# minimum and maximum gate size constraint for all gates
constraints.extend([gate_sizes[i] >= 1 for i in range(N)])
constraints.extend([gate_sizes[i] <= gate_size_max for i in range(N)])

# define the problem
model = Model(np.sum(gate_sizes), constraints)
solution = model.solve(verbosity = 1)

# extract the results
T_arrival = solution['variables']['my_T']
gate_sizes = solution['variables']['gs']

# print the results
print(f'T_arrival = {round(T_arrival, 2)}')
print(f'T_arrival (ps) = {round(T_arrival * tau, 2)}')
print(f'Gate Sizes: {gate_sizes}')

Using solver 'mosek_cli'
 for 7 free variables
  in 17 posynomial inequalities.
Solving took 0.021 seconds.
T_arrival = 26.5
T_arrival (ps) = 132.49
Gate Sizes: [1.         1.00000001 1.00000001 1.09353583 2.22138106 8.74441316]
